In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: D:\ID2223 Scalable Machine Learning\Lab 1\mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
import importlib
from mlfs.airquality import util
importlib.reload(util)
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-17 09:06:23,302 INFO: Initializing external client
2025-11-17 09:06:23,303 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 09:06:25,701 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286306


'{"country": "sweden", "city": "stockholm", "street": "st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": "59.3251172", "longitude": "18.0710935"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_rolling',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [5]:
air_quality_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 


,date,pm25,rolling1,rolling2,rolling3,country,city,street,url
0,2024-04-06 00:00:00+00:00,14.0,16.0,14.0,13.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
1,2020-06-16 00:00:00+00:00,25.0,18.0,16.0,15.666667,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2,2019-01-28 00:00:00+00:00,44.0,45.0,44.0,51.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
3,2019-02-21 00:00:00+00:00,36.0,39.0,50.0,49.666668,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
4,2021-10-27 00:00:00+00:00,12.0,35.0,29.5,25.333334,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
...,...,...,...,...,...,...,...,...,...
2458,2025-11-13 00:00:00+00:00,9.0,18.0,19.5,18.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2459,2025-11-15 00:00:00+00:00,12.0,13.0,11.0,13.333333,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2460,2025-11-16 00:00:00+00:00,7.0,12.0,12.5,11.333333,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2461,2025-11-14 00:00:00+00:00,13.0,9.0,13.5,16.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [6]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
# aq_today_df = aq_today_df.drop(['country','city','street','url'],axis = 'columns')
aq_today_df

,pm25,country,city,street,date,url
0,15.0,sweden,stockholm,st-eriksgatan-83,2025-11-17,https://api.waqi.info/feed/@10523


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-17,0.40,0.0,11.457958,316.272949,stockholm
1,2025-11-18,-0.55,0.0,8.089993,290.854523,stockholm
2,2025-11-19,-0.20,0.6,10.028439,338.962402,stockholm
3,2025-11-20,-0.50,0.0,14.561099,8.530692,stockholm
4,2025-11-21,-4.15,0.0,12.387348,305.537750,stockholm
5,2025-11-22,-2.85,0.0,1.800000,216.869980,stockholm
6,2025-11-23,1.20,0.0,9.178235,78.690102,stockholm
7,2025-11-24,2.55,0.2,10.948973,207.407486,stockholm


In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')
hourly_df

Coordinates 59.25°N 18.0°E
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
date,,,,
2025-11-17 00:00:00,-1.30,0.0,6.792466,327.994659
2025-11-17 01:00:00,-1.55,0.0,7.594208,328.570496
2025-11-17 02:00:00,-1.75,0.0,8.587338,326.976135
2025-11-17 03:00:00,-1.90,0.0,9.387651,327.528839
2025-11-17 04:00:00,-2.10,0.0,9.021574,331.389618
...,...,...,...,...
2025-11-24 19:00:00,0.90,0.0,8.788720,214.992096
2025-11-24 20:00:00,0.95,0.0,9.000000,216.869980
2025-11-24 21:00:00,0.95,0.0,9.504272,217.304031


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
csv_file=f"{root_dir}/data/air-quality-data.csv"
util.check_file_path(csv_file)
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
latest_day = df['date'].max()
# latest_day = pd.to_datetime(latest_day)  # Keep it timezone-naive
latest_day = pd.to_datetime(latest_day)
latest_day

File successfully found at the path: D:\ID2223 Scalable Machine Learning\Lab 1\mlfs-book/data/air-quality-data.csv


Timestamp('2025-11-16 00:00:00')

In [11]:
# New data from the latest date in .csv file
aq_df_pseu = air_quality_fg.read()
aq_df_pseu['date'] = aq_df_pseu['date'].dt.tz_localize(None)
filtered_df = aq_df_pseu[(aq_df_pseu['date']>= latest_day + datetime.timedelta(days = -3))]
# Drop the unnecessary columns for rolling
filtered_df = filtered_df.drop(['rolling1','rolling2','rolling3','country','city','street','url'],axis = 'columns')
filtered_df = pd.concat([filtered_df,aq_today_df.drop(['country','city','street','url'],axis = 'columns')],ignore_index=True)
filtered_df = filtered_df.set_index('date').sort_index()
filtered_df.insert(1,'rolling1',filtered_df.rolling(1).mean()['pm25'].shift(1))
filtered_df.insert(1,'rolling2',filtered_df.rolling(2).mean()['pm25'].shift(1))
filtered_df.insert(1,'rolling3',filtered_df.rolling(3).mean()['pm25'].shift(1))

filtered_df = filtered_df.reset_index()
filtered_df = filtered_df[filtered_df['date'] > latest_day]
filtered_df['country']=country
filtered_df['city']=city
filtered_df['street']=street
filtered_df['url']=aqicn_url
filtered_df['pm25'] = filtered_df['pm25'].astype('float32')
filtered_df['rolling1'] = filtered_df['rolling1'].astype('float32')
filtered_df['rolling2'] = filtered_df['rolling2'].astype('float32')
filtered_df['rolling3'] = filtered_df['rolling3'].astype('float32')
filtered_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.94s) 


,date,pm25,rolling3,rolling2,rolling1,country,city,street,url
4,2025-11-17,7.0,10.666667,9.5,7.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
5,2025-11-17,15.0,8.666667,7.0,7.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523


In [12]:
# Insert new data
air_quality_fg.insert(filtered_df)

2025-11-17 09:06:34,943 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286306/fs/1273933/fg/1721712


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 2/2 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_rolling_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286306/jobs/named/air_quality_rolling_1_offline_fg_materialization/executions


(Job('air_quality_rolling_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761879
         }
       },
       "result": {
         "observed_value": 7.0,
         "element_count": 2,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T08:06:34.000943Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_e

In [13]:
# air_quality_fg.read().set_index('date').sort_index()
air_quality_fg.read().set_index('date').sort_index()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.25s) 


,pm25,rolling1,rolling2,rolling3,country,city,street,url
date,,,,,,,,
2018-04-12 00:00:00+00:00,26.0,23.0,18.5,30.666666,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-13 00:00:00+00:00,33.0,26.0,24.5,21.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-14 00:00:00+00:00,37.0,33.0,29.5,27.333334,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-15 00:00:00+00:00,64.0,37.0,35.0,32.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-16 00:00:00+00:00,70.0,64.0,50.5,44.666668,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
...,...,...,...,...,...,...,...,...
2025-11-13 00:00:00+00:00,9.0,18.0,19.5,18.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2025-11-14 00:00:00+00:00,13.0,9.0,13.5,16.000000,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2025-11-15 00:00:00+00:00,12.0,13.0,11.0,13.333333,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523


In [14]:
air_quality_fg.read()[air_quality_fg.read()['rolling3'].isna()].set_index('date').sort_index()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.74s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 


,pm25,rolling1,rolling2,rolling3,country,city,street,url
date,,,,,,,,


In [15]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-17 09:07:14,624 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286306/fs/1273933/fg/1637985


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 8/8 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286306/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-17 09:07:31,274 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 09:07:34,580 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 09:09:56,990 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 09:09:57,161 INFO: Waiting for log aggregation to finish.
2025-11-17 09:10:06,023 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735327
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 8,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T08:07:14.000624Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

In [16]:
air = air_quality_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.42s) 


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
